In [39]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import *
import requests
import re
from tqdm import trange
from lxml.html import parse
import numpy as np
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')
#options.add_argument('window-size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36')
driver = webdriver.Chrome('chromedriver', chrome_options=options)
#driver.set_window_size(1080,720)
driver.implicitly_wait(2)
driver.get('https://google.com')

In [3]:
elem=driver.find_element_by_id('lst-ib')
elem.clear()
elem.send_keys('autopilot')
elem.submit()

In [4]:
tab = driver.find_elements_by_css_selector('a.q.qs')
for i in tab:
    if ('뉴스' in i.text):
        newstab=i.get_attribute('href')
        driver.get(newstab)
        break

In [5]:
pagenum=10
article_urls=[]
if (driver.find_element_by_css_selector('td.cur').text=='1'):
    article = driver.find_elements_by_css_selector('a.l.lLrAF')           
    article_urls = [i.get_attribute('href') for i in article]
for i in trange(pagenum):
    page=driver.find_elements_by_css_selector('a.fl')
    for j in page:
        if j.text==str(i+1):
            alpha=j.get_attribute('href')
            driver.get(alpha)
            article = driver.find_elements_by_css_selector('a.l.lLrAF')           
            article_urls.extend([k.get_attribute('href') for k in article])
            break

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  5.77it/s]

 20%|██        | 2/10 [00:00<00:03,  2.05it/s]

 30%|███       | 3/10 [00:01<00:03,  1.97it/s]

 40%|████      | 4/10 [00:02<00:03,  1.83it/s]

...

 60%|██████    | 6/10 [00:03<00:02,  1.69it/s]

 70%|███████   | 7/10 [00:04<00:01,  1.63it/s]

 80%|████████  | 8/10 [00:05<00:01,  1.59it/s]

 90%|█████████ | 9/10 [00:05<00:00,  1.57it/s]

100%|██████████| 10/10 [00:06<00:00,  1.53it/s]

In [ ]:
article_urls[0:len(article_urls)/10]

['https://www.ttnews.com/articles/tesla-public-or-private-still-will-release-autopilot-safety-data',
 'https://www.businessinsider.com/tesla-starts-offering-owners-free-autopilot-trials-2018-8',
 'https://www.forbes.com/sites/davidsilver/2018/08/15/tesla-model-3-autopilot-outperforms-model-s-competitors-in-lane-keeping-test/',
 'https://www.sfchronicle.com/business/article/Tesla-public-or-private-will-still-release-13159080.php',
 'https://www.theverge.com/2018/8/1/17641186/tesla-elon-musk-self-driving-coast-to-coast-delay',
 'https://interestingengineering.com/elon-musk-reveals-tesla-autopilot-30-will-be-free-with-5000-full-self-driving-pack',
 'https://electrek.co/2018/07/28/tesla-model-3-autopilot-avoid-crash-nearmiss-dashcam/',
 'https://insideevs.com/get-your-free-tesla-autopilot-trial-now/',
 'https://electrek.co/2018/08/08/tesla-autopilot-hardware-upgrade-free-with-full-self-driving-package/',
 'https://www.techrepublic.com/article/teslas-autopilot-cheat-sheet/',
 'https://www.t

In [7]:
len(article_urls)

100

In [16]:
c=[]
for i in trange(len(article_urls)):
    try:
        driver.get(article_urls[i])
        try:
            atext=driver.find_elements_by_css_selector('p')
            a=[j.text for j in atext]
        except StaleElementReferenceException:
            atext=driver.find_elements_by_css_selector('p')
            a=[j.text for j in atext]
        b=''
        for k in range(0,len(a)):
            b=b+a[k]
        if(len(b)<=100):
            driver.implicitly_wait(5)
            try:
                atext=driver.find_elements_by_css_selector('p')
                a=[j.text for j in atext]
            except StaleElementReferenceException:
                atext=driver.find_elements_by_css_selector('p')
                a=[j.text for j in atext]
            b=''
            for k in range(0,len(a)):
                b=b+a[k]
            if(len(b)<=100):
                int('Error')
        c.append(b)
    except ValueError:
        try:
            response = requests.get(article_urls[i])
            html = response.text
            soup=bs(html,'html.parser')
            b=''
            for tit in soup.find_all('p')[1:]:
                b=b+str(tit.find(text=True))
            c.append(b)
        except ValueError:
            error='ValueError'+' '+article_urls[i]
            c.append(error)

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:03<05:45,  3.49s/it]

  2%|▏         | 2/100 [00:09<07:54,  4.84s/it]

  3%|▎         | 3/100 [00:25<13:51,  8.57s/it]

  4%|▍         | 4/100 [00:34<13:44,  8.58s/it]

...

 96%|█████████▌| 96/100 [21:03<00:52, 13.16s/it]

 97%|█████████▋| 97/100 [21:13<00:39, 13.13s/it]

 98%|█████████▊| 98/100 [21:16<00:26, 13.03s/it]

 99%|█████████▉| 99/100 [21:46<00:13, 13.20s/it]

100%|██████████| 100/100 [21:58<00:00, 13.18s/it]

In [65]:
c[52]

'Nach tödlichen Unfällen mit Teslas "Autopilot" stellt sich manchen die Frage nach der Sicherheit von Assistenzsystemen. In einer US-Studie schneidet vor allem ein Modell gut ab. Doch die Tester haben offenbar etwas Wichtiges übersehen.Wie sicher sind die fortgeschrittenen Fahrassistenzsysteme in Modellen der Oberklasse? Ein Test des unabhängigen Insurance Institute for Highway Safety (IIHS) in den USA hat ergeben, dass nicht alle getesteten Modelle die Erwartungen gleichermaßen erfüllen. Das gelte sowohl für sogenannte Abstandsregeltempomaten als auch für aktive Spurhalteassistenten. Allerdings scheinen die Tester nicht berücksichtigt zu haben, dass beispielsweise die Mercedes-Modelle manche Funktion bewusst einschränken.Das IIHS untersuchte für den Test einen 5er BMW, eine Mercedes E-Klasse, einen Volvo S90 sowie die Tesla-Modelle S und 3. Allerdings verfügte das Model S nur über die Autopilot-Version 7.1, während das neuere Model 3 die verbesserte Version 8.1 nutzte.Recht zuverlässi

In [47]:
for i in range(0,len(c)):
    c[i]=re.sub('\n|\r',' ',c[i])

In [19]:
driver.quit()

In [51]:
data=np.array([article_urls,c])

IndexError: index 8 is out of bounds for axis 0 with size 2

In [60]:
pd.DataFrame(data.T,columns=['URL','Article']).to_csv('Google-autopilot.csv',index=False)